In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp "/content/drive/MyDrive/TwiBot-20/Twibot-20.zip" "data.zip"
!unzip data.zip

Archive:  data.zip
   creating: Twibot-20/
  inflating: Twibot-20/dev.json      
  inflating: Twibot-20/LICENSE       
  inflating: Twibot-20/README.txt    
  inflating: Twibot-20/support.json  
  inflating: Twibot-20/test.json     
  inflating: Twibot-20/train.json    
  inflating: Twibot-20/TwiBot-20 Seed Users.txt  


In [119]:
import pandas as pd
import numpy as np
import random

In [120]:
import re
import numpy as np
import pandas as pd
# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# nltk
from nltk.stem import WordNetLemmatizer
# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [121]:
train = pd.read_json('/content/Twibot-20/train.json')
test = pd.read_json('/content/Twibot-20/test.json')
#support = pd.read_json('/content/Twibot-20/support.json')
#dev = pd.read_json('/content/Twibot-20/dev.json')

In [122]:
dev = pd.read_json('/content/Twibot-20/dev.json')

In [123]:
train

,ID,profile,tweet,neighbor,domain,label
0,17461978,"{'id': '17461978 ', 'id_str': '17461978 ', 'na...",[RT @CarnivalCruise: 🎉 Are you ready to see wh...,None,"[Politics, Business, Entertainment]",0
1,1297437077403885568,"{'id': '1297437077403885568 ', 'id_str': '1297...",None,"{'following': ['170861207', '23970102', '47293...",[Politics],1
2,17685258,"{'id': '17685258 ', 'id_str': '17685258 ', 'na...",[RT @realDonaldTrump: THANK YOU #RNC2020! http...,"{'following': ['46464108', '21536398', '186434...","[Politics, Entertainment, Sports]",0
3,15750898,"{'id': '15750898 ', 'id_str': '15750898 ', 'na...",[A family fears they may have been cheated out...,"{'following': ['2324715174', '24030137', '2336...",[Politics],0
4,1659167666,"{'id': '1659167666 ', 'id_str': '1659167666 ',...",[RT @VonteThePlug: Yeah but he ain’t got one h...,"{'following': ['1628313708', '726405625', '130...",[Politics],1
...,...,...,...,...,...,...
8273,1630890068,"{'id': '1630890068 ', 'id_str': '1630890068 ',...",[@sethgoldberg17 @jaysonst Fan interference? I...,"{'following': ['237453978', '462581299', '1706...",[Sports],0
8274,713519580757536768,"{'id': '713519580757536769 ', 'id_str': '71351...",[@C130Matt I think I heard a voice from out in...,"{'following': ['36991422', '32567081', '133983...",[Sports],1
8275,93345260,"{'id': '93345260 ', 'id_str': '93345260 ', 'na...",[@savage_esquire That's unfuckingbelievable.\n...,"{'following': ['714636670268792832', '23341114...",[Sports],1
8276,1749309397,"{'id': '1749309397 ', 'id_str': '1749309397 ',...",[@Jomboy_ Doesn’t want to pull anymore Hammys\...,"{'following': ['3124065581', '413364940', '211...",[Sports],1


In [124]:
test

,ID,profile,tweet,neighbor,domain,label
0,1188812492010487808,"{'id': '1188812492010487808 ', 'id_str': '1188...",[RT @clevelanddotcom: Three Ohio House Republi...,None,"[Politics, Sports]",1
1,155659213,"{'id': '155659213 ', 'id_str': '155659213 ', '...","[We touch our hair 96 times a day on average, ...",None,"[Politics, Business, Entertainment, Sports]",0
2,147725246,"{'id': '147725246 ', 'id_str': '147725246 ', '...",['He Looked Like He Knew What He Was Doing': C...,"{'following': ['36734275', '20713061', '755419...",[Politics],0
3,1296248637194895360,"{'id': '1296248637194895360 ', 'id_str': '1296...","[Estamos abiertos a colaboraciones, por lo cuá...","{'following': ['87818409', '41390292', '140910...",[Politics],1
4,1339835893,"{'id': '1339835893 ', 'id_str': '1339835893 ',...",[The suffragists chose purple and gold to repr...,None,"[Politics, Business, Entertainment, Sports]",0
...,...,...,...,...,...,...
1178,452754350,"{'id': '452754350 ', 'id_str': '452754350 ', '...",[Chicago Cubs sign [former Texas Tech] relieve...,"{'following': ['2924422992', '2365623499', '33...",[Sports],1
1179,850435801687183360,"{'id': '850435801687183362 ', 'id_str': '85043...",[RT @TheWoodofHolly: @CardsFromAttic GMA is su...,"{'following': ['704144006129692674', '95336330...",[Sports],1
1180,2188795745,"{'id': '2188795745 ', 'id_str': '2188795745 ',...",[RT @OriginalFunko: RT &amp; follow @OriginalF...,"{'following': ['66762778', '2981733093', '1861...",[Sports],1
1181,940687680,"{'id': '940687680 ', 'id_str': '940687680 ', '...",[RT @pierre_rochard: Run the numbers.\n\nBuy B...,"{'following': ['559791853', '10080654991362498...",[Sports],1


In [125]:
training = train[['profile', 'tweet', 'label']]
testing = test[['profile', 'tweet', 'label']]

In [126]:
training

,profile,tweet,label
0,"{'id': '17461978 ', 'id_str': '17461978 ', 'na...",[RT @CarnivalCruise: 🎉 Are you ready to see wh...,0
1,"{'id': '1297437077403885568 ', 'id_str': '1297...",None,1
2,"{'id': '17685258 ', 'id_str': '17685258 ', 'na...",[RT @realDonaldTrump: THANK YOU #RNC2020! http...,0
3,"{'id': '15750898 ', 'id_str': '15750898 ', 'na...",[A family fears they may have been cheated out...,0
4,"{'id': '1659167666 ', 'id_str': '1659167666 ',...",[RT @VonteThePlug: Yeah but he ain’t got one h...,1
...,...,...,...
8273,"{'id': '1630890068 ', 'id_str': '1630890068 ',...",[@sethgoldberg17 @jaysonst Fan interference? I...,0
8274,"{'id': '713519580757536769 ', 'id_str': '71351...",[@C130Matt I think I heard a voice from out in...,1
8275,"{'id': '93345260 ', 'id_str': '93345260 ', 'na...",[@savage_esquire That's unfuckingbelievable.\n...,1
8276,"{'id': '1749309397 ', 'id_str': '1749309397 ',...",[@Jomboy_ Doesn’t want to pull anymore Hammys\...,1


In [127]:
testing

,profile,tweet,label
0,"{'id': '1188812492010487808 ', 'id_str': '1188...",[RT @clevelanddotcom: Three Ohio House Republi...,1
1,"{'id': '155659213 ', 'id_str': '155659213 ', '...","[We touch our hair 96 times a day on average, ...",0
2,"{'id': '147725246 ', 'id_str': '147725246 ', '...",['He Looked Like He Knew What He Was Doing': C...,0
3,"{'id': '1296248637194895360 ', 'id_str': '1296...","[Estamos abiertos a colaboraciones, por lo cuá...",1
4,"{'id': '1339835893 ', 'id_str': '1339835893 ',...",[The suffragists chose purple and gold to repr...,0
...,...,...,...
1178,"{'id': '452754350 ', 'id_str': '452754350 ', '...",[Chicago Cubs sign [former Texas Tech] relieve...,1
1179,"{'id': '850435801687183362 ', 'id_str': '85043...",[RT @TheWoodofHolly: @CardsFromAttic GMA is su...,1
1180,"{'id': '2188795745 ', 'id_str': '2188795745 ',...",[RT @OriginalFunko: RT &amp; follow @OriginalF...,1
1181,"{'id': '940687680 ', 'id_str': '940687680 ', '...",[RT @pierre_rochard: Run the numbers.\n\nBuy B...,1


In [128]:
training_0 = training.dropna(axis = 0)
testing_0 = testing.dropna(axis = 0)

In [129]:
training_0

,profile,tweet,label
0,"{'id': '17461978 ', 'id_str': '17461978 ', 'na...",[RT @CarnivalCruise: 🎉 Are you ready to see wh...,0
2,"{'id': '17685258 ', 'id_str': '17685258 ', 'na...",[RT @realDonaldTrump: THANK YOU #RNC2020! http...,0
3,"{'id': '15750898 ', 'id_str': '15750898 ', 'na...",[A family fears they may have been cheated out...,0
4,"{'id': '1659167666 ', 'id_str': '1659167666 ',...",[RT @VonteThePlug: Yeah but he ain’t got one h...,1
5,"{'id': '34743251 ', 'id_str': '34743251 ', 'na...","[RT @elonmusk: Aloha, welcome back from space ...",0
...,...,...,...
8273,"{'id': '1630890068 ', 'id_str': '1630890068 ',...",[@sethgoldberg17 @jaysonst Fan interference? I...,0
8274,"{'id': '713519580757536769 ', 'id_str': '71351...",[@C130Matt I think I heard a voice from out in...,1
8275,"{'id': '93345260 ', 'id_str': '93345260 ', 'na...",[@savage_esquire That's unfuckingbelievable.\n...,1
8276,"{'id': '1749309397 ', 'id_str': '1749309397 ',...",[@Jomboy_ Doesn’t want to pull anymore Hammys\...,1


In [130]:
training_0.reset_index(drop=True, inplace=True)

In [131]:
training_0

,profile,tweet,label
0,"{'id': '17461978 ', 'id_str': '17461978 ', 'na...",[RT @CarnivalCruise: 🎉 Are you ready to see wh...,0
1,"{'id': '17685258 ', 'id_str': '17685258 ', 'na...",[RT @realDonaldTrump: THANK YOU #RNC2020! http...,0
2,"{'id': '15750898 ', 'id_str': '15750898 ', 'na...",[A family fears they may have been cheated out...,0
3,"{'id': '1659167666 ', 'id_str': '1659167666 ',...",[RT @VonteThePlug: Yeah but he ain’t got one h...,1
4,"{'id': '34743251 ', 'id_str': '34743251 ', 'na...","[RT @elonmusk: Aloha, welcome back from space ...",0
...,...,...,...
8218,"{'id': '1630890068 ', 'id_str': '1630890068 ',...",[@sethgoldberg17 @jaysonst Fan interference? I...,0
8219,"{'id': '713519580757536769 ', 'id_str': '71351...",[@C130Matt I think I heard a voice from out in...,1
8220,"{'id': '93345260 ', 'id_str': '93345260 ', 'na...",[@savage_esquire That's unfuckingbelievable.\n...,1
8221,"{'id': '1749309397 ', 'id_str': '1749309397 ',...",[@Jomboy_ Doesn’t want to pull anymore Hammys\...,1


In [135]:
testing_0.reset_index(drop=True, inplace=True)

In [136]:
training_0

,profile,tweet,label
0,"{'id': '17461978 ', 'id_str': '17461978 ', 'na...",[RT @CarnivalCruise: 🎉 Are you ready to see wh...,0
1,"{'id': '17685258 ', 'id_str': '17685258 ', 'na...",[RT @realDonaldTrump: THANK YOU #RNC2020! http...,0
2,"{'id': '15750898 ', 'id_str': '15750898 ', 'na...",[A family fears they may have been cheated out...,0
3,"{'id': '1659167666 ', 'id_str': '1659167666 ',...",[RT @VonteThePlug: Yeah but he ain’t got one h...,1
4,"{'id': '34743251 ', 'id_str': '34743251 ', 'na...","[RT @elonmusk: Aloha, welcome back from space ...",0
...,...,...,...
8218,"{'id': '1630890068 ', 'id_str': '1630890068 ',...",[@sethgoldberg17 @jaysonst Fan interference? I...,0
8219,"{'id': '713519580757536769 ', 'id_str': '71351...",[@C130Matt I think I heard a voice from out in...,1
8220,"{'id': '93345260 ', 'id_str': '93345260 ', 'na...",[@savage_esquire That's unfuckingbelievable.\n...,1
8221,"{'id': '1749309397 ', 'id_str': '1749309397 ',...",[@Jomboy_ Doesn’t want to pull anymore Hammys\...,1


In [137]:
train_profile = pd.json_normalize(training_0['profile'])
test_profile = pd.json_normalize(testing_0['profile'])

In [138]:
train_profile

,id,id_str,name,screen_name,location,profile_location,description,url,entities,protected,...,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image
0,17461978,17461978,SHAQ,SHAQ,"Orlando, FL","{'id': '55b4f9e5c516e0b6', 'url': 'https://api...","VERY QUOTATIOUS, I PERFORM RANDOM ACTS OF SHAQ...",https://t.co/7hsiK8cCKW,{'url': {'urls': [{'url': 'https://t.co/7hsiK8...,False,...,http://pbs.twimg.com/profile_images/1673907275...,https://pbs.twimg.com/profile_images/167390727...,2FC2EF,181A1E,252429,666666,True,False,False,False
1,17685258,17685258,Brad Parscale,parscale,Florida,None,Owner @ Parscale Strategy. Senior Advisor Digi...,https://t.co/GooZcYDqFg,{'url': {'urls': [{'url': 'https://t.co/GooZcY...,False,...,http://pbs.twimg.com/profile_images/1295453225...,https://pbs.twimg.com/profile_images/129545322...,AB2316,FFFFFF,FFFFFF,666666,False,False,False,False
2,15750898,15750898,FOX 13 Tampa Bay,FOX13News,"Tampa, FL",None,Bringing you the important stuff like breaking...,https://t.co/RtP9QYEZCq,{'url': {'urls': [{'url': 'https://t.co/RtP9QY...,False,...,http://pbs.twimg.com/profile_images/1293193013...,https://pbs.twimg.com/profile_images/129319301...,0B2F8A,FFFFFF,E8EEF0,333333,True,False,False,False
3,1659167666,1659167666,Vonte The Plug 🎤🔌,VonteThePlugNC,"Jacksonville Beach, FL","{'id': '5e281c17a74c170f', 'url': 'https://api...",MOTIVATION 3 OUT NOW 🔥 Singles: ‘Lil Shawdy’ &...,https://t.co/5cY8GWvk8E,{'url': {'urls': [{'url': 'https://t.co/5cY8GW...,False,...,http://pbs.twimg.com/profile_images/1181662400...,https://pbs.twimg.com/profile_images/118166240...,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False
4,34743251,34743251,SpaceX,SpaceX,"Hawthorne, CA",None,"SpaceX designs, manufactures and launches the ...",https://t.co/SDnmlLwwoK,{'url': {'urls': [{'url': 'https://t.co/SDnmlL...,False,...,http://pbs.twimg.com/profile_images/1082744382...,https://pbs.twimg.com/profile_images/108274438...,62616B,FFFFFF,EFEFEF,333333,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8218,1630890068,1630890068,Mike Waters,MikeWatersSYR,,None,College basketball writer at https://t.co/OOrF...,None,{'description': {'urls': [{'url': 'https://t.c...,False,...,http://pbs.twimg.com/profile_images/6543578675...,https://pbs.twimg.com/profile_images/654357867...,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False
8219,713519580757536769,713519580757536769,Shelby Pitre,FairWarning66,,None,"My wife is my best friend, dad, Grandpa, 10th ...",None,{'description': {'urls': []}},False,...,http://pbs.twimg.com/profile_images/1122683499...,https://pbs.twimg.com/profile_images/112268349...,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False
8220,93345260,93345260,Mateo Cristiani,MateoCristiani,"Florida, USA",None,I was a professional circus acrobat for 27 yea...,None,{'description': {'urls': []}},False,...,http://pbs.twimg.com/profile_images/8224563194...,https://pbs.twimg.com/profile_images/822456319...,1DA1F2,C0DEED,DDEEF6,333333,True,True,True,False
8221,1749309397,1749309397,Old Ant,notes114,"New York, USA",None,"Part time project mgt geek, full time concert ...",None,{'description': {'urls': []}},False,...,http://pbs.twimg.com/profile_images/3788000004...,https://pbs.twimg.com/profile_images/378800000...,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False


In [139]:
test_profile

,id,id_str,name,screen_name,location,profile_location,description,url,entities,protected,...,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image
0,1188812492010487808,1188812492010487808,Sharon Israel ⭐️⭐️⭐️,SharonIsrael10,Los Angeles & Colorado,None,Day 1 Trump supporter. I rode the escalator! C...,None,{'description': {'urls': []}},False,...,http://pbs.twimg.com/profile_images/1188836420...,https://pbs.twimg.com/profile_images/118883642...,1DA1F2,C0DEED,DDEEF6,333333,True,True,True,False
1,155659213,155659213,Cristiano Ronaldo,Cristiano,"Turim, Piemonte",None,This Privacy Policy addresses the collection a...,https://t.co/VcAXB9ryty,{'url': {'urls': [{'url': 'https://t.co/VcAXB9...,False,...,http://pbs.twimg.com/profile_images/1157313327...,https://pbs.twimg.com/profile_images/115731332...,1643C9,FFFFFF,838387,0D0D0D,True,False,False,False
2,147725246,147725246,FoxNewsInsider,FoxNewsInsider,NYC,None,Stay connected with everything Fox - the lates...,http://t.co/Y7kRySoYtY,{'url': {'urls': [{'url': 'http://t.co/Y7kRySo...,False,...,http://pbs.twimg.com/profile_images/8819320209...,https://pbs.twimg.com/profile_images/881932020...,0084B4,FFFFFF,DDEEF6,333333,True,False,False,False
3,1296248637194895360,1296248637194895360,El Realista,ElReali03271594,Puerto Rico,"{'id': '48c0e174396d2cbf', 'url': 'https://api...",Aprendizaje. Pensamiento Crítico. Debate de id...,None,{'description': {'urls': []}},False,...,http://pbs.twimg.com/profile_images/1296249309...,https://pbs.twimg.com/profile_images/129624930...,1DA1F2,C0DEED,DDEEF6,333333,True,True,True,False
4,1339835893,1339835893,Hillary Clinton,HillaryClinton,"New York, NY",None,"2016 Democratic Nominee, SecState, Senator, ha...",https://t.co/99DVW9q1yt,{'url': {'urls': [{'url': 'https://t.co/99DVW9...,False,...,http://pbs.twimg.com/profile_images/1291192333...,https://pbs.twimg.com/profile_images/129119233...,0057B8,000000,000000,000000,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,452754350,452754350,Alan Reifman,AlanReifman,"Lubbock, Texas",None,Texas Tech professor of human devt and family ...,http://t.co/vwXejeH3i0,{'url': {'urls': [{'url': 'http://t.co/vwXejeH...,False,...,http://pbs.twimg.com/profile_images/4718599734...,https://pbs.twimg.com/profile_images/471859973...,0084B4,FFFFFF,DDEEF6,333333,True,False,False,False
1169,850435801687183362,850435801687183362,Junk Wax Investment Services ($19.99 Per Month),CardsFromAttic,JunkWaxSylvania,None,Satirizing the sports card industry one tweet ...,https://t.co/CzOhDBbE8U,{'url': {'urls': [{'url': 'https://t.co/CzOhDB...,False,...,http://pbs.twimg.com/profile_images/1300649646...,https://pbs.twimg.com/profile_images/130064964...,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False
1170,2188795745,2188795745,B,bkgreen09,United States,None,,None,{'description': {'urls': []}},False,...,http://pbs.twimg.com/profile_images/1168654618...,https://pbs.twimg.com/profile_images/116865461...,3B94D9,000000,000000,000000,False,True,False,False
1171,940687680,940687680,bilal koç,bilalko14,,None,,None,{'description': {'urls': []}},False,...,http://pbs.twimg.com/profile_images/2840708085...,https://pbs.twimg.com/profile_images/284070808...,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False


In [140]:
from datetime import datetime

def time_conversion(x):
    x = datetime.strftime(datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y '), '%Y-%m-%d %H:%M:%S')
    x = (pd.Timestamp.now() - pd.Timestamp(x))
    return (int(round(x.total_seconds() / 3600)))


In [141]:
train_profile['user_age'] = train_profile['created_at'].apply(time_conversion)
test_profile['user_age'] = test_profile['created_at'].apply(time_conversion)

In [142]:
train_profile.head()

,id,id_str,name,screen_name,location,profile_location,description,url,entities,protected,...,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,user_age
0,17461978,17461978,SHAQ,SHAQ,"Orlando, FL","{'id': '55b4f9e5c516e0b6', 'url': 'https://api...","VERY QUOTATIOUS, I PERFORM RANDOM ACTS OF SHAQ...",https://t.co/7hsiK8cCKW,{'url': {'urls': [{'url': 'https://t.co/7hsiK8...,False,...,https://pbs.twimg.com/profile_images/167390727...,2FC2EF,181A1E,252429,666666,True,False,False,False,118210
1,17685258,17685258,Brad Parscale,parscale,Florida,None,Owner @ Parscale Strategy. Senior Advisor Digi...,https://t.co/GooZcYDqFg,{'url': {'urls': [{'url': 'https://t.co/GooZcY...,False,...,https://pbs.twimg.com/profile_images/129545322...,AB2316,FFFFFF,FFFFFF,666666,False,False,False,False,117985
2,15750898,15750898,FOX 13 Tampa Bay,FOX13News,"Tampa, FL",None,Bringing you the important stuff like breaking...,https://t.co/RtP9QYEZCq,{'url': {'urls': [{'url': 'https://t.co/RtP9QY...,False,...,https://pbs.twimg.com/profile_images/129319301...,0B2F8A,FFFFFF,E8EEF0,333333,True,False,False,False,120701
3,1659167666,1659167666,Vonte The Plug 🎤🔌,VonteThePlugNC,"Jacksonville Beach, FL","{'id': '5e281c17a74c170f', 'url': 'https://api...",MOTIVATION 3 OUT NOW 🔥 Singles: ‘Lil Shawdy’ &...,https://t.co/5cY8GWvk8E,{'url': {'urls': [{'url': 'https://t.co/5cY8GW...,False,...,https://pbs.twimg.com/profile_images/118166240...,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False,76793
4,34743251,34743251,SpaceX,SpaceX,"Hawthorne, CA",None,"SpaceX designs, manufactures and launches the ...",https://t.co/SDnmlLwwoK,{'url': {'urls': [{'url': 'https://t.co/SDnmlL...,False,...,https://pbs.twimg.com/profile_images/108274438...,62616B,FFFFFF,EFEFEF,333333,True,False,False,False,114454


In [143]:
test_profile.head()

,id,id_str,name,screen_name,location,profile_location,description,url,entities,protected,...,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,user_age
0,1188812492010487808,1188812492010487808,Sharon Israel ⭐️⭐️⭐️,SharonIsrael10,Los Angeles & Colorado,None,Day 1 Trump supporter. I rode the escalator! C...,None,{'description': {'urls': []}},False,...,https://pbs.twimg.com/profile_images/118883642...,1DA1F2,C0DEED,DDEEF6,333333,True,True,True,False,22302
1,155659213,155659213,Cristiano Ronaldo,Cristiano,"Turim, Piemonte",None,This Privacy Policy addresses the collection a...,https://t.co/VcAXB9ryty,{'url': {'urls': [{'url': 'https://t.co/VcAXB9...,False,...,https://pbs.twimg.com/profile_images/115731332...,1643C9,FFFFFF,838387,0D0D0D,True,False,False,False,104449
2,147725246,147725246,FoxNewsInsider,FoxNewsInsider,NYC,None,Stay connected with everything Fox - the lates...,http://t.co/Y7kRySoYtY,{'url': {'urls': [{'url': 'http://t.co/Y7kRySo...,False,...,https://pbs.twimg.com/profile_images/881932020...,0084B4,FFFFFF,DDEEF6,333333,True,False,False,False,104950
3,1296248637194895360,1296248637194895360,El Realista,ElReali03271594,Puerto Rico,"{'id': '48c0e174396d2cbf', 'url': 'https://api...",Aprendizaje. Pensamiento Crítico. Debate de id...,None,{'description': {'urls': []}},False,...,https://pbs.twimg.com/profile_images/129624930...,1DA1F2,C0DEED,DDEEF6,333333,True,True,True,False,15187
4,1339835893,1339835893,Hillary Clinton,HillaryClinton,"New York, NY",None,"2016 Democratic Nominee, SecState, Senator, ha...",https://t.co/99DVW9q1yt,{'url': {'urls': [{'url': 'https://t.co/99DVW9...,False,...,https://pbs.twimg.com/profile_images/129119233...,0057B8,000000,000000,000000,False,True,False,False,79730


In [144]:
train_profile['statuses_count'] = pd.to_numeric(train_profile['statuses_count'])
train_profile['followers_count'] = pd.to_numeric(train_profile['followers_count'])
train_profile['friends_count'] = pd.to_numeric(train_profile['friends_count'])
train_profile['favourites_count'] = pd.to_numeric(train_profile['favourites_count'])
train_profile['listed_count'] = pd.to_numeric(train_profile['listed_count'])



In [145]:
test_profile['statuses_count'] = pd.to_numeric(test_profile['statuses_count'])
test_profile['followers_count'] = pd.to_numeric(test_profile['followers_count'])
test_profile['friends_count'] = pd.to_numeric(test_profile['friends_count'])
test_profile['favourites_count'] = pd.to_numeric(test_profile['favourites_count'])
test_profile['listed_count'] = pd.to_numeric(test_profile['listed_count'])

In [146]:
train_profile['friends_count'] = np.where(train_profile['friends_count'] == 0, 1, train_profile['friends_count'])

In [147]:
test_profile['friends_count'] = np.where(test_profile['friends_count'] == 0, 1, test_profile['friends_count'])

In [148]:
train_profile['tweet_freq'] = train_profile['statuses_count']/train_profile['user_age']
train_profile['followers_growth_rate'] = train_profile['followers_count']/train_profile['user_age']
train_profile['friends_growth_rate'] = train_profile['friends_count']/train_profile['user_age']
train_profile['favourites_growth_rate'] = train_profile['favourites_count']/train_profile['user_age']
train_profile['listed_growth_rate'] = train_profile['listed_count']/train_profile['user_age']
train_profile['followers_friends_ratio'] = train_profile['followers_count']/train_profile['friends_count']
train_profile['screen_name_length'] = train_profile['screen_name'].apply(lambda x: len(x))
train_profile['num_digits_in_screen_name'] = train_profile['screen_name'].apply(lambda x: sum(c.isdigit() for c in x))
train_profile['name_length'] = train_profile['name'].apply(lambda x: len(x))
train_profile['num_digits_in_name'] = train_profile['name'].apply(lambda x: sum(c.isdigit() for c in x))
train_profile['description_length'] = train_profile['description'].apply(lambda x: len(x))

In [149]:
test_profile['tweet_freq'] = test_profile['statuses_count']/test_profile['user_age']
test_profile['followers_growth_rate'] = test_profile['followers_count']/test_profile['user_age']
test_profile['friends_growth_rate'] = test_profile['friends_count']/test_profile['user_age']
test_profile['favourites_growth_rate'] = test_profile['favourites_count']/test_profile['user_age']
test_profile['listed_growth_rate'] = test_profile['listed_count']/test_profile['user_age']
test_profile['followers_friends_ratio'] = test_profile['followers_count']/test_profile['friends_count']
test_profile['screen_name_length'] = test_profile['screen_name'].apply(lambda x: len(x))
test_profile['num_digits_in_screen_name'] = test_profile['screen_name'].apply(lambda x: sum(c.isdigit() for c in x))
test_profile['name_length'] = test_profile['name'].apply(lambda x: len(x))
test_profile['num_digits_in_name'] = test_profile['name'].apply(lambda x: sum(c.isdigit() for c in x))
test_profile['description_length'] = test_profile['description'].apply(lambda x: len(x))

In [150]:
features = ['default_profile', 'favourites_count', 'followers_count', 'friends_count', 'geo_enabled', 'listed_count', 'profile_use_background_image', 'statuses_count', 'verified', 'tweet_freq', 'followers_growth_rate', 'friends_growth_rate', 'favourites_growth_rate', 'listed_growth_rate', 'followers_friends_ratio', 'screen_name_length', 'num_digits_in_screen_name', 'name_length', 'num_digits_in_name', 'description_length']
len(features)

20

In [151]:
train_profile = train_profile[features]


In [152]:
train_profile.head()

,default_profile,favourites_count,followers_count,friends_count,geo_enabled,listed_count,profile_use_background_image,statuses_count,verified,tweet_freq,followers_growth_rate,friends_growth_rate,favourites_growth_rate,listed_growth_rate,followers_friends_ratio,screen_name_length,num_digits_in_screen_name,name_length,num_digits_in_name,description_length
0,False,142,15349596,692,True,45568,True,9798,True,0.082886,129.850233,0.005854,0.001201,0.385483,22181.497110,5,0,5,0,51
1,False,953,762839,475,True,3201,False,5518,True,0.046769,6.465559,0.004026,0.008077,0.027131,1605.976842,9,0,14,0,161
2,False,2946,327587,4801,True,1744,True,192876,True,1.597965,2.714037,0.039776,0.024407,0.014449,68.233076,10,2,17,2,161
3,True,729,13324,647,False,44,True,103,False,0.001341,0.173505,0.008425,0.009493,0.000573,20.593509,15,0,18,0,104
4,False,177,12601567,96,False,26952,True,4879,True,0.042628,110.101587,0.000839,0.001546,0.235483,131266.322917,7,0,7,0,91


In [153]:
train_profile.shape

(8223, 20)

In [154]:
test_profile = test_profile[features]
test_profile.head()

,default_profile,favourites_count,followers_count,friends_count,geo_enabled,listed_count,profile_use_background_image,statuses_count,verified,tweet_freq,followers_growth_rate,friends_growth_rate,favourites_growth_rate,listed_growth_rate,followers_friends_ratio,screen_name_length,num_digits_in_screen_name,name_length,num_digits_in_name,description_length
0,True,42684,16596,16944,False,1,True,49757,False,2.231056,0.744149,0.759752,1.913909,0.000045,9.794618e-01,15,2,21,0,158
1,False,2,87313765,50,True,83703,True,3569,True,0.034170,835.946395,0.000479,0.000019,0.801377,1.746275e+06,10,0,18,0,103
2,False,762,161827,361,False,1471,True,73786,True,0.703059,1.541944,0.003440,0.007261,0.014016,4.482742e+02,15,0,15,0,149
3,True,0,9,543,False,0,True,2,False,0.000132,0.000593,0.035754,0.000000,0.000000,1.657459e-02,16,8,12,0,146
4,False,1123,28513011,846,False,40146,False,11945,True,0.149818,357.619604,0.010611,0.014085,0.503524,3.370332e+04,15,0,16,0,155


In [155]:
test_profile.shape

(1173, 20)

In [156]:
#Convert object features to binary
func = {'False ': 0, 'True ': 1}
train_profile['default_profile'] = train_profile['default_profile'].map(func)
train_profile['geo_enabled'] = train_profile['geo_enabled'].map(func)
train_profile['profile_use_background_image'] = train_profile['profile_use_background_image'].map(func)
train_profile['verified'] = train_profile['verified'].map(func)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [157]:
#Convert object features to binary

test_profile['default_profile'] = test_profile['default_profile'].map(func)
test_profile['geo_enabled'] = test_profile['geo_enabled'].map(func)
test_profile['profile_use_background_image'] = test_profile['profile_use_background_image'].map(func)
test_profile['verified'] = test_profile['verified'].map(func)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [158]:
train_profile

,default_profile,favourites_count,followers_count,friends_count,geo_enabled,listed_count,profile_use_background_image,statuses_count,verified,tweet_freq,followers_growth_rate,friends_growth_rate,favourites_growth_rate,listed_growth_rate,followers_friends_ratio,screen_name_length,num_digits_in_screen_name,name_length,num_digits_in_name,description_length
0,0,142,15349596,692,1,45568,1,9798,1,0.082886,129.850233,0.005854,0.001201,0.385483,22181.497110,5,0,5,0,51
1,0,953,762839,475,1,3201,0,5518,1,0.046769,6.465559,0.004026,0.008077,0.027131,1605.976842,9,0,14,0,161
2,0,2946,327587,4801,1,1744,1,192876,1,1.597965,2.714037,0.039776,0.024407,0.014449,68.233076,10,2,17,2,161
3,1,729,13324,647,0,44,1,103,0,0.001341,0.173505,0.008425,0.009493,0.000573,20.593509,15,0,18,0,104
4,0,177,12601567,96,0,26952,1,4879,1,0.042628,110.101587,0.000839,0.001546,0.235483,131266.322917,7,0,7,0,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8218,1,20115,10997,821,0,221,1,25790,1,0.334648,0.142696,0.010653,0.261010,0.002868,13.394641,14,0,12,0,161
8219,1,298,44,637,0,0,1,425,0,0.007903,0.000818,0.011845,0.005541,0.000000,0.069074,14,2,13,0,158
8220,1,2464,342,849,0,0,1,1439,0,0.013179,0.003132,0.007776,0.022566,0.000000,0.402827,15,0,16,0,134
8221,1,2634,72,367,0,2,1,674,0,0.008856,0.000946,0.004822,0.034608,0.000026,0.196185,9,3,8,0,66


In [159]:
test_profile

,default_profile,favourites_count,followers_count,friends_count,geo_enabled,listed_count,profile_use_background_image,statuses_count,verified,tweet_freq,followers_growth_rate,friends_growth_rate,favourites_growth_rate,listed_growth_rate,followers_friends_ratio,screen_name_length,num_digits_in_screen_name,name_length,num_digits_in_name,description_length
0,1,42684,16596,16944,0,1,1,49757,0,2.231056,0.744149,0.759752,1.913909,0.000045,9.794618e-01,15,2,21,0,158
1,0,2,87313765,50,1,83703,1,3569,1,0.034170,835.946395,0.000479,0.000019,0.801377,1.746275e+06,10,0,18,0,103
2,0,762,161827,361,0,1471,1,73786,1,0.703059,1.541944,0.003440,0.007261,0.014016,4.482742e+02,15,0,15,0,149
3,1,0,9,543,0,0,1,2,0,0.000132,0.000593,0.035754,0.000000,0.000000,1.657459e-02,16,8,12,0,146
4,0,1123,28513011,846,0,40146,0,11945,1,0.149818,357.619604,0.010611,0.014085,0.503524,3.370332e+04,15,0,16,0,155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,0,75,7760,8104,0,106,1,2330,0,0.025646,0.085412,0.089198,0.000826,0.001167,9.575518e-01,12,0,13,0,153
1169,1,125825,8446,408,0,55,1,50168,0,1.122025,0.188898,0.009125,2.814121,0.001230,2.070098e+01,15,0,48,4,144
1170,0,1572,309,1961,1,3,0,2950,0,0.039685,0.004157,0.026380,0.021147,0.000040,1.575727e-01,10,2,2,0,1
1171,1,627,154,1019,0,0,1,152,0,0.001824,0.001848,0.012230,0.007525,0.000000,1.511286e-01,10,2,10,0,1


In [160]:
training_0

,profile,tweet,label
0,"{'id': '17461978 ', 'id_str': '17461978 ', 'na...",[RT @CarnivalCruise: 🎉 Are you ready to see wh...,0
1,"{'id': '17685258 ', 'id_str': '17685258 ', 'na...",[RT @realDonaldTrump: THANK YOU #RNC2020! http...,0
2,"{'id': '15750898 ', 'id_str': '15750898 ', 'na...",[A family fears they may have been cheated out...,0
3,"{'id': '1659167666 ', 'id_str': '1659167666 ',...",[RT @VonteThePlug: Yeah but he ain’t got one h...,1
4,"{'id': '34743251 ', 'id_str': '34743251 ', 'na...","[RT @elonmusk: Aloha, welcome back from space ...",0
...,...,...,...
8218,"{'id': '1630890068 ', 'id_str': '1630890068 ',...",[@sethgoldberg17 @jaysonst Fan interference? I...,0
8219,"{'id': '713519580757536769 ', 'id_str': '71351...",[@C130Matt I think I heard a voice from out in...,1
8220,"{'id': '93345260 ', 'id_str': '93345260 ', 'na...",[@savage_esquire That's unfuckingbelievable.\n...,1
8221,"{'id': '1749309397 ', 'id_str': '1749309397 ',...",[@Jomboy_ Doesn’t want to pull anymore Hammys\...,1


In [162]:
train_tweet = training_0[['tweet']]
test_tweet = testing_0[['tweet']]

In [163]:
train_tweet

,tweet
0,[RT @CarnivalCruise: 🎉 Are you ready to see wh...
1,[RT @realDonaldTrump: THANK YOU #RNC2020! http...
2,[A family fears they may have been cheated out...
3,[RT @VonteThePlug: Yeah but he ain’t got one h...
4,"[RT @elonmusk: Aloha, welcome back from space ..."
...,...
8218,[@sethgoldberg17 @jaysonst Fan interference? I...
8219,[@C130Matt I think I heard a voice from out in...
8220,[@savage_esquire That's unfuckingbelievable.\n...
8221,[@Jomboy_ Doesn’t want to pull anymore Hammys\...


In [164]:
train_tweet.loc[:, 'tweet_mod'] = ""
for i in range(len(train_tweet['tweet'])):
  l = train_tweet['tweet'][i]
  temp = ""
  for j in range(len(l)):
    text = l[j]
    temp = temp + text
  train_tweet.loc[i, 'tweet_mod'] = train_tweet.loc[i, 'tweet_mod'] + temp

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [165]:
train_tweet

,tweet,tweet_mod
0,[RT @CarnivalCruise: 🎉 Are you ready to see wh...,RT @CarnivalCruise: 🎉 Are you ready to see wha...
1,[RT @realDonaldTrump: THANK YOU #RNC2020! http...,RT @realDonaldTrump: THANK YOU #RNC2020! https...
2,[A family fears they may have been cheated out...,A family fears they may have been cheated out ...
3,[RT @VonteThePlug: Yeah but he ain’t got one h...,RT @VonteThePlug: Yeah but he ain’t got one ha...
4,"[RT @elonmusk: Aloha, welcome back from space ...","RT @elonmusk: Aloha, welcome back from space 💫..."
...,...,...
8218,[@sethgoldberg17 @jaysonst Fan interference? I...,@sethgoldberg17 @jaysonst Fan interference? Im...
8219,[@C130Matt I think I heard a voice from out in...,@C130Matt I think I heard a voice from out in ...
8220,[@savage_esquire That's unfuckingbelievable.\n...,@savage_esquire That's unfuckingbelievable.\nI...
8221,[@Jomboy_ Doesn’t want to pull anymore Hammys\...,@Jomboy_ Doesn’t want to pull anymore Hammys\n...


In [166]:
test_tweet.loc[:, 'tweet_mod'] = ""
for i in range(len(test_tweet['tweet'])):
  l = test_tweet['tweet'][i]
  temp = ""
  for j in range(len(l)):
    text = l[j]
    temp = temp + text
  test_tweet.loc[i, 'tweet_mod'] = test_tweet.loc[i, 'tweet_mod'] + temp

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [167]:
test_tweet

,tweet,tweet_mod
0,[RT @clevelanddotcom: Three Ohio House Republi...,RT @clevelanddotcom: Three Ohio House Republic...
1,"[We touch our hair 96 times a day on average, ...","We touch our hair 96 times a day on average, t..."
2,['He Looked Like He Knew What He Was Doing': C...,'He Looked Like He Knew What He Was Doing': CA...
3,"[Estamos abiertos a colaboraciones, por lo cuá...","Estamos abiertos a colaboraciones, por lo cuál..."
4,[The suffragists chose purple and gold to repr...,The suffragists chose purple and gold to repre...
...,...,...
1168,[Chicago Cubs sign [former Texas Tech] relieve...,Chicago Cubs sign [former Texas Tech] reliever...
1169,[RT @TheWoodofHolly: @CardsFromAttic GMA is su...,RT @TheWoodofHolly: @CardsFromAttic GMA is sup...
1170,[RT @OriginalFunko: RT &amp; follow @OriginalF...,RT @OriginalFunko: RT &amp; follow @OriginalFu...
1171,[RT @pierre_rochard: Run the numbers.\n\nBuy B...,RT @pierre_rochard: Run the numbers.\n\nBuy Bi...


In [168]:
train_tweet = train_tweet[['tweet_mod']]
test_tweet = test_tweet[['tweet_mod']]

In [169]:
train_tweet

,tweet_mod
0,RT @CarnivalCruise: 🎉 Are you ready to see wha...
1,RT @realDonaldTrump: THANK YOU #RNC2020! https...
2,A family fears they may have been cheated out ...
3,RT @VonteThePlug: Yeah but he ain’t got one ha...
4,"RT @elonmusk: Aloha, welcome back from space 💫..."
...,...
8218,@sethgoldberg17 @jaysonst Fan interference? Im...
8219,@C130Matt I think I heard a voice from out in ...
8220,@savage_esquire That's unfuckingbelievable.\nI...
8221,@Jomboy_ Doesn’t want to pull anymore Hammys\n...


In [170]:
test_tweet

,tweet_mod
0,RT @clevelanddotcom: Three Ohio House Republic...
1,"We touch our hair 96 times a day on average, t..."
2,'He Looked Like He Knew What He Was Doing': CA...
3,"Estamos abiertos a colaboraciones, por lo cuál..."
4,The suffragists chose purple and gold to repre...
...,...
1168,Chicago Cubs sign [former Texas Tech] reliever...
1169,RT @TheWoodofHolly: @CardsFromAttic GMA is sup...
1170,RT @OriginalFunko: RT &amp; follow @OriginalFu...
1171,RT @pierre_rochard: Run the numbers.\n\nBuy Bi...


In [171]:
def extract_hash_tags(s):
  return len(list(part[1:] for part in s.split() if part.startswith('#')))

In [172]:
def getNoDuplicateHashtags(s):
  MyList = list(part[1:] for part in s.split() if part.startswith('#'))
  res = {}
  sum = 0
  for i in MyList:
    res[i] = MyList.count(i)
    sum += res[i]
  return len(res)

In [173]:
def getCountDuplicateHashtags(s):
  MyList = list(part[1:] for part in s.split() if part.startswith('#'))
  res = {}
  sum = 0
  for i in MyList:
    res[i] = MyList.count(i)
    sum += res[i]
  return len(res)

In [174]:
emoji_regex = re.compile('[#*0-9]️⃣|[©®‼⁉™ℹ↔-↙↩↪⌚⌛⌨⏏⏩-⏳⏸-⏺Ⓜ▪▫▶◀◻-◾☀-☄☎☑☔☕☘]|☝[🏻-🏿]?|[☠☢☣☦☪☮☯☸-☺♀♂♈-♓♟♠♣♥♦♨♻♾♿⚒-⚗⚙⚛⚜⚠⚡⚪⚫⚰⚱⚽⚾⛄⛅⛈⛎⛏⛑⛓⛔⛩⛪⛰-⛵⛷⛸]|⛹(?:️‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[⛺⛽✂✅✈✉]|[✊-✍][🏻-🏿]?|[✏✒✔✖✝✡✨✳✴❄❇❌❎❓-❕❗❣❤➕-➗➡➰➿⤴⤵⬅-⬇⬛⬜⭐⭕〰〽㊗㊙🀄🃏🅰🅱🅾🅿🆎🆑-🆚]|🇦[🇨-🇬🇮🇱🇲🇴🇶-🇺🇼🇽🇿]|🇧[🇦🇧🇩-🇯🇱-🇴🇶-🇹🇻🇼🇾🇿]|🇨[🇦🇨🇩🇫-🇮🇰-🇵🇷🇺-🇿]|🇩[🇪🇬🇯🇰🇲🇴🇿]|🇪[🇦🇨🇪🇬🇭🇷-🇺]|🇫[🇮-🇰🇲🇴🇷]|🇬[🇦🇧🇩-🇮🇱-🇳🇵-🇺🇼🇾]|🇭[🇰🇲🇳🇷🇹🇺]|🇮[🇨-🇪🇱-🇴🇶-🇹]|🇯[🇪🇲🇴🇵]|🇰[🇪🇬-🇮🇲🇳🇵🇷🇼🇾🇿]|🇱[🇦-🇨🇮🇰🇷-🇻🇾]|🇲[🇦🇨-🇭🇰-🇿]|🇳[🇦🇨🇪-🇬🇮🇱🇴🇵🇷🇺🇿]|🇴🇲|🇵[🇦🇪-🇭🇰-🇳🇷-🇹🇼🇾]|🇶🇦|🇷[🇪🇴🇸🇺🇼]|🇸[🇦-🇪🇬-🇴🇷-🇹🇻🇽-🇿]|🇹[🇦🇨🇩🇫-🇭🇯-🇴🇷🇹🇻🇼🇿]|🇺[🇦🇬🇲🇳🇸🇾🇿]|🇻[🇦🇨🇪🇬🇮🇳🇺]|🇼[🇫🇸]|🇽🇰|🇾[🇪🇹]|🇿[🇦🇲🇼]|[🈁🈂🈚🈯🈲-🈺🉐🉑🌀-🌡🌤-🎄]|🎅[🏻-🏿]?|[🎆-🎓🎖🎗🎙-🎛🎞-🏁]|🏂[🏻-🏿]?|[🏃🏄](?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[🏅🏆]|🏇[🏻-🏿]?|[🏈🏉]|🏊(?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[🏋🏌](?:️‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[🏍-🏰]|🏳(?:️‍🌈)?|🏴(?:‍☠️|󠁧󠁢(?:󠁥󠁮󠁧|󠁳󠁣󠁴|󠁷󠁬󠁳)󠁿)?|[🏵🏷-👀]|👁(?:️‍🗨️)?|[👂👃][🏻-🏿]?|[👄👅]|[👆-👐][🏻-🏿]?|[👑-👥]|[👦👧][🏻-🏿]?|👨(?:‍(?:[⚕⚖✈]️|❤️‍(?:💋‍)?👨|[🌾🍳🎓🎤🎨🏫🏭]|👦(?:‍👦)?|👧(?:‍[👦👧])?|[👨👩]‍(?:👦(?:‍👦)?|👧(?:‍[👦👧])?)|[💻💼🔧🔬🚀🚒🦰-🦳])|[🏻-🏿](?:‍(?:[⚕⚖✈]️|[🌾🍳🎓🎤🎨🏫🏭💻💼🔧🔬🚀🚒🦰-🦳]))?)?|👩(?:‍(?:[⚕⚖✈]️|❤️‍(?:💋‍)?[👨👩]|[🌾🍳🎓🎤🎨🏫🏭]|👦(?:‍👦)?|👧(?:‍[👦👧])?|👩‍(?:👦(?:‍👦)?|👧(?:‍[👦👧])?)|[💻💼🔧🔬🚀🚒🦰-🦳])|[🏻-🏿](?:‍(?:[⚕⚖✈]️|[🌾🍳🎓🎤🎨🏫🏭💻💼🔧🔬🚀🚒🦰-🦳]))?)?|[👪-👭]|👮(?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|👯(?:‍[♀♂]️)?|👰[🏻-🏿]?|👱(?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|👲[🏻-🏿]?|👳(?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[👴-👶][🏻-🏿]?|👷(?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|👸[🏻-🏿]?|[👹-👻]|👼[🏻-🏿]?|[👽-💀]|[💁💂](?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|💃[🏻-🏿]?|💄|💅[🏻-🏿]?|[💆💇](?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[💈-💩]|💪[🏻-🏿]?|[💫-📽📿-🔽🕉-🕎🕐-🕧🕯🕰🕳]|🕴[🏻-🏿]?|🕵(?:️‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[🕶-🕹]|🕺[🏻-🏿]?|[🖇🖊-🖍]|[🖐🖕🖖][🏻-🏿]?|[🖤🖥🖨🖱🖲🖼🗂-🗄🗑-🗓🗜-🗞🗡🗣🗨🗯🗳🗺-🙄]|[🙅-🙇](?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[🙈-🙊]|🙋(?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|🙌[🏻-🏿]?|[🙍🙎](?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|🙏[🏻-🏿]?|[🚀-🚢]|🚣(?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[🚤-🚳]|[🚴-🚶](?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[🚷-🚿]|🛀[🏻-🏿]?|[🛁-🛅🛋]|🛌[🏻-🏿]?|[🛍-🛒🛠-🛥🛩🛫🛬🛰🛳-🛹🤐-🤗]|[🤘-🤜][🏻-🏿]?|🤝|[🤞🤟][🏻-🏿]?|[🤠-🤥]|🤦(?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[🤧-🤯]|[🤰-🤶][🏻-🏿]?|🤷(?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[🤸🤹](?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|🤺|🤼(?:‍[♀♂]️)?|[🤽🤾](?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[🥀-🥅🥇-🥰🥳-🥶🥺🥼-🦢🦰-🦴]|[🦵🦶][🏻-🏿]?|🦷|[🦸🦹](?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[🧀-🧂🧐]|[🧑-🧕][🏻-🏿]?|🧖(?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[🧗-🧝](?:‍[♀♂]️|[🏻-🏿](?:‍[♀♂]️)?)?|[🧞🧟](?:‍[♀♂]️)?|[🧠-🧿]')
def getNoOfEmojis(s):
  return len(re.findall(emoji_regex, s))

In [175]:
def getNoOfUrls(s):
  return len(re.findall(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", s))

In [176]:
train_tweet = pd.concat([train_tweet, training_0['label']], axis = 1)
test_tweet = pd.concat([test_tweet, testing_0['label']], axis = 1)

In [177]:
train_tweet

,tweet_mod,label
0,RT @CarnivalCruise: 🎉 Are you ready to see wha...,0
1,RT @realDonaldTrump: THANK YOU #RNC2020! https...,0
2,A family fears they may have been cheated out ...,0
3,RT @VonteThePlug: Yeah but he ain’t got one ha...,1
4,"RT @elonmusk: Aloha, welcome back from space 💫...",0
...,...,...
8218,@sethgoldberg17 @jaysonst Fan interference? Im...,0
8219,@C130Matt I think I heard a voice from out in ...,1
8220,@savage_esquire That's unfuckingbelievable.\nI...,1
8221,@Jomboy_ Doesn’t want to pull anymore Hammys\n...,1


In [178]:
test_tweet

,tweet_mod,label
0,RT @clevelanddotcom: Three Ohio House Republic...,1
1,"We touch our hair 96 times a day on average, t...",0
2,'He Looked Like He Knew What He Was Doing': CA...,0
3,"Estamos abiertos a colaboraciones, por lo cuál...",1
4,The suffragists chose purple and gold to repre...,0
...,...,...
1168,Chicago Cubs sign [former Texas Tech] reliever...,1
1169,RT @TheWoodofHolly: @CardsFromAttic GMA is sup...,1
1170,RT @OriginalFunko: RT &amp; follow @OriginalFu...,1
1171,RT @pierre_rochard: Run the numbers.\n\nBuy Bi...,1


In [179]:
train_tweet['no_of_hashtags'] = train_tweet['tweet_mod'].apply(extract_hash_tags)
test_tweet['no_of_hashtags'] = test_tweet['tweet_mod'].apply(extract_hash_tags)
train_tweet['no_of_emojis'] = train_tweet['tweet_mod'].apply(getNoOfEmojis)
test_tweet['no_of_emojis'] = test_tweet['tweet_mod'].apply(getNoOfEmojis)
train_tweet['no_of_urls'] = train_tweet['tweet_mod'].apply(getNoOfUrls)
test_tweet['no_of_urls'] = test_tweet['tweet_mod'].apply(getNoOfUrls)

In [180]:
train_tweet

,tweet_mod,label,no_of_hashtags,no_of_emojis,no_of_urls
0,RT @CarnivalCruise: 🎉 Are you ready to see wha...,0,152,73,207
1,RT @realDonaldTrump: THANK YOU #RNC2020! https...,0,36,69,93
2,A family fears they may have been cheated out ...,0,60,28,173
3,RT @VonteThePlug: Yeah but he ain’t got one ha...,1,6,108,57
4,"RT @elonmusk: Aloha, welcome back from space 💫...",0,9,8,178
...,...,...,...,...,...
8218,@sethgoldberg17 @jaysonst Fan interference? Im...,0,1,7,99
8219,@C130Matt I think I heard a voice from out in ...,1,8,1,13
8220,@savage_esquire That's unfuckingbelievable.\nI...,1,109,51,73
8221,@Jomboy_ Doesn’t want to pull anymore Hammys\n...,1,2,23,4


In [181]:
train_tweet['no_of_hashtags_avg'] = train_tweet['no_of_hashtags'].mean()
test_tweet['no_of_hashtags_avg'] = test_tweet['no_of_hashtags'].mean()

train_tweet['no_of_hashtags_std'] = train_tweet['no_of_hashtags'].std()
test_tweet['no_of_hashtags_std'] = test_tweet['no_of_hashtags'].std()

train_tweet['no_of_urls_avg'] = train_tweet['no_of_urls'].mean()
test_tweet['no_of_urls_avg'] = test_tweet['no_of_urls'].mean()

train_tweet['no_of_urls_std'] = train_tweet['no_of_urls'].std()
test_tweet['no_of_urls_std'] = test_tweet['no_of_urls'].std()

train_tweet['no_of_emojis_avg'] = train_tweet['no_of_emojis'].mean()
test_tweet['no_of_emojis_avg'] = test_tweet['no_of_emojis'].mean()

train_tweet['no_of_emojis_std'] = train_tweet['no_of_emojis'].std()
test_tweet['no_of_emojis_std'] = test_tweet['no_of_emojis'].std()

In [182]:
train_profile.columns

Index(['default_profile', 'favourites_count', 'followers_count',
       'friends_count', 'geo_enabled', 'listed_count',
       'profile_use_background_image', 'statuses_count', 'verified',
       'tweet_freq', 'followers_growth_rate', 'friends_growth_rate',
       'favourites_growth_rate', 'listed_growth_rate',
       'followers_friends_ratio', 'screen_name_length',
       'num_digits_in_screen_name', 'name_length', 'num_digits_in_name',
       'description_length'],
      dtype='object')

In [183]:
train_profile = pd.concat([train_profile, training_0['label']], axis = 1)
test_profile = pd.concat([test_profile, testing_0['label']], axis = 1)

In [184]:
train_profile.columns

Index(['default_profile', 'favourites_count', 'followers_count',
       'friends_count', 'geo_enabled', 'listed_count',
       'profile_use_background_image', 'statuses_count', 'verified',
       'tweet_freq', 'followers_growth_rate', 'friends_growth_rate',
       'favourites_growth_rate', 'listed_growth_rate',
       'followers_friends_ratio', 'screen_name_length',
       'num_digits_in_screen_name', 'name_length', 'num_digits_in_name',
       'description_length', 'label'],
      dtype='object')

In [185]:
train_tweet.columns

Index(['tweet_mod', 'label', 'no_of_hashtags', 'no_of_emojis', 'no_of_urls',
       'no_of_hashtags_avg', 'no_of_hashtags_std', 'no_of_urls_avg',
       'no_of_urls_std', 'no_of_emojis_avg', 'no_of_emojis_std'],
      dtype='object')

In [186]:
Tr = pd.concat([train_profile, train_tweet[['no_of_hashtags_avg', 'no_of_hashtags_std', 'no_of_emojis_avg', 'no_of_emojis_std', 'no_of_urls_avg', 'no_of_urls_std']]], axis = 1)
te = pd.concat([test_profile, test_tweet[['no_of_hashtags_avg', 'no_of_hashtags_std', 'no_of_emojis_avg', 'no_of_emojis_std', 'no_of_urls_avg', 'no_of_urls_std']]], axis = 1)

In [239]:
Tr = pd.concat([train_profile, train_tweet[['no_of_hashtags', 'no_of_emojis', 'no_of_urls']]], axis = 1)
te = pd.concat([test_profile, test_tweet[['no_of_hashtags', 'no_of_emojis', 'no_of_urls']]], axis = 1)

In [240]:
Tr.columns

Index(['default_profile', 'favourites_count', 'followers_count',
       'friends_count', 'geo_enabled', 'listed_count',
       'profile_use_background_image', 'statuses_count', 'verified',
       'tweet_freq', 'followers_growth_rate', 'friends_growth_rate',
       'favourites_growth_rate', 'listed_growth_rate',
       'followers_friends_ratio', 'screen_name_length',
       'num_digits_in_screen_name', 'name_length', 'num_digits_in_name',
       'description_length', 'label', 'no_of_hashtags', 'no_of_emojis',
       'no_of_urls'],
      dtype='object')

In [241]:
new_all = pd.concat([Tr, te], axis = 0)

In [249]:
features = ['default_profile', 'favourites_count', 'followers_count', 'friends_count', 'geo_enabled', 'listed_count', 'profile_use_background_image', 'statuses_count', 'verified', 'tweet_freq', 'followers_growth_rate', 'friends_growth_rate', 'favourites_growth_rate', 'listed_growth_rate', 'followers_friends_ratio', 'screen_name_length', 'num_digits_in_screen_name', 'name_length', 'num_digits_in_name', 'description_length']
len(features)

20

In [250]:
new_all.columns

Index(['default_profile', 'favourites_count', 'followers_count',
       'friends_count', 'geo_enabled', 'listed_count',
       'profile_use_background_image', 'statuses_count', 'verified',
       'tweet_freq', 'followers_growth_rate', 'friends_growth_rate',
       'favourites_growth_rate', 'listed_growth_rate',
       'followers_friends_ratio', 'screen_name_length',
       'num_digits_in_screen_name', 'name_length', 'num_digits_in_name',
       'description_length', 'label', 'no_of_hashtags', 'no_of_emojis',
       'no_of_urls'],
      dtype='object')

In [251]:
features.extend(['no_of_hashtags', 'no_of_emojis', 'no_of_urls'])

In [252]:
from sklearn import svm, tree
import xgboost
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [253]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix
new_all[features] = scaler.fit_transform(new_all[features])
X_train, X_test, y_train, y_test = train_test_split(new_all[features], new_all['label'], test_size=0.5, random_state=1, stratify=new_all['label'])

In [254]:
xgb = xgboost.XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print('Accuracy: %.4f' % accuracy_score(y_test, y_pred))

Accuracy: 0.8244


In [259]:
train_profile

,default_profile,favourites_count,followers_count,friends_count,geo_enabled,listed_count,profile_use_background_image,statuses_count,verified,tweet_freq,...,friends_growth_rate,favourites_growth_rate,listed_growth_rate,followers_friends_ratio,screen_name_length,num_digits_in_screen_name,name_length,num_digits_in_name,description_length,label
0,0,142,15349596,692,1,45568,1,9798,1,0.082886,...,0.005854,0.001201,0.385483,22181.497110,5,0,5,0,51,0
1,0,953,762839,475,1,3201,0,5518,1,0.046769,...,0.004026,0.008077,0.027131,1605.976842,9,0,14,0,161,0
2,0,2946,327587,4801,1,1744,1,192876,1,1.597965,...,0.039776,0.024407,0.014449,68.233076,10,2,17,2,161,0
3,1,729,13324,647,0,44,1,103,0,0.001341,...,0.008425,0.009493,0.000573,20.593509,15,0,18,0,104,1
4,0,177,12601567,96,0,26952,1,4879,1,0.042628,...,0.000839,0.001546,0.235483,131266.322917,7,0,7,0,91,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8218,1,20115,10997,821,0,221,1,25790,1,0.334648,...,0.010653,0.261010,0.002868,13.394641,14,0,12,0,161,0
8219,1,298,44,637,0,0,1,425,0,0.007903,...,0.011845,0.005541,0.000000,0.069074,14,2,13,0,158,1
8220,1,2464,342,849,0,0,1,1439,0,0.013179,...,0.007776,0.022566,0.000000,0.402827,15,0,16,0,134,1
8221,1,2634,72,367,0,2,1,674,0,0.008856,...,0.004822,0.034608,0.000026,0.196185,9,3,8,0,66,1


In [260]:
bots_prof = train_profile[train_profile['label'] == 1].reset_index(drop=True)
genuine_prof = train_profile[train_profile['label'] == 0].reset_index(drop = True)

In [261]:
bots_prof_te = test_profile[test_profile['label'] == 1].reset_index(drop=True)
genuine_prof_te = test_profile[test_profile['label'] == 0].reset_index(drop = True)

In [262]:
bots_prof

,default_profile,favourites_count,followers_count,friends_count,geo_enabled,listed_count,profile_use_background_image,statuses_count,verified,tweet_freq,...,friends_growth_rate,favourites_growth_rate,listed_growth_rate,followers_friends_ratio,screen_name_length,num_digits_in_screen_name,name_length,num_digits_in_name,description_length,label
0,1,729,13324,647,0,44,1,103,0,0.001341,...,0.008425,0.009493,0.000573,20.593509,15,0,18,0,104,1
1,1,0,4,66,0,0,1,0,0,0.000000,...,0.004372,0.000000,0.000000,0.060606,16,8,16,0,58,1
2,1,2224,13120,1406,0,235,1,44434,0,0.526569,...,0.016662,0.026356,0.002785,9.331437,13,0,14,0,58,1
3,0,382,2829,789,0,31,1,211695,0,1.856941,...,0.006921,0.003351,0.000272,3.585551,12,0,6,0,50,1
4,1,1419,648,513,0,7,1,542,0,0.006014,...,0.005692,0.015745,0.000078,1.263158,14,0,15,0,43,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4626,1,66,2378,457,0,49,1,24894,0,0.258870,...,0.004752,0.000686,0.000510,5.203501,10,0,12,0,107,1
4627,1,298,44,637,0,0,1,425,0,0.007903,...,0.011845,0.005541,0.000000,0.069074,14,2,13,0,158,1
4628,1,2464,342,849,0,0,1,1439,0,0.013179,...,0.007776,0.022566,0.000000,0.402827,15,0,16,0,134,1
4629,1,2634,72,367,0,2,1,674,0,0.008856,...,0.004822,0.034608,0.000026,0.196185,9,3,8,0,66,1


In [263]:
genuine_prof

,default_profile,favourites_count,followers_count,friends_count,geo_enabled,listed_count,profile_use_background_image,statuses_count,verified,tweet_freq,...,friends_growth_rate,favourites_growth_rate,listed_growth_rate,followers_friends_ratio,screen_name_length,num_digits_in_screen_name,name_length,num_digits_in_name,description_length,label
0,0,142,15349596,692,1,45568,1,9798,1,0.082886,...,0.005854,0.001201,0.385483,22181.497110,5,0,5,0,51,0
1,0,953,762839,475,1,3201,0,5518,1,0.046769,...,0.004026,0.008077,0.027131,1605.976842,9,0,14,0,161,0
2,0,2946,327587,4801,1,1744,1,192876,1,1.597965,...,0.039776,0.024407,0.014449,68.233076,10,2,17,2,161,0
3,0,177,12601567,96,0,26952,1,4879,1,0.042628,...,0.000839,0.001546,0.235483,131266.322917,7,0,7,0,91,0
4,0,4532,112403156,295385,0,550181,0,31055,1,0.269849,...,2.566713,0.039380,4.780732,380.531022,13,0,14,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3587,0,2431,14141,769,0,211,1,15400,1,0.143579,...,0.007170,0.022665,0.001967,18.388817,15,0,16,0,85,0
3588,1,5774,127,540,1,0,1,4053,0,0.245116,...,0.032658,0.349199,0.000000,0.235185,10,0,5,0,128,0
3589,0,88,8676,131,0,266,1,5721,1,0.055864,...,0.001279,0.000859,0.002597,66.229008,11,0,15,0,13,0
3590,0,25867,34008,15566,1,719,1,11177,1,0.096061,...,0.133783,0.222315,0.006179,2.184762,13,0,15,0,158,0


In [264]:
tr_prof = pd.concat([bots_prof, genuine_prof], axis = 0)
te_prof = pd.concat([bots_prof_te, genuine_prof_te], axis = 0)

In [265]:
bots_train = train_tweet[train_tweet['label'] == 1].reset_index(drop=True)
genuine_train = train_tweet[train_tweet['label'] == 0].reset_index(drop = True)

In [266]:
bots_test = test_tweet[test_tweet['label'] == 1].reset_index(drop=True)
genuine_test = test_tweet[test_tweet['label'] == 0].reset_index(drop = True)

In [267]:
bots_train

,tweet_mod,label,no_of_hashtags,no_of_emojis,no_of_urls,no_of_hashtags_avg,no_of_hashtags_std,no_of_urls_avg,no_of_urls_std,no_of_emojis_avg,no_of_emojis_std
0,RT @VonteThePlug: Yeah but he ain’t got one ha...,1,6,108,57,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
1,https://t.co/KWnKV9F0fw\n,1,0,0,1,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
2,RT @toadmeister: If the BBC is investing £100 ...,1,33,21,92,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
3,@BrightskinMacky 😂😂😂\nBone what??? 😂😂😂 RT @sun...,1,16,93,14,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
4,"Milano pigliatutto. Dopo l’ex Banco Popolare, ...",1,124,16,141,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
...,...,...,...,...,...,...,...,...,...,...,...
4626,Love &amp; Entrepreneurship: Relationship Ther...,1,30,22,161,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
4627,@C130Matt I think I heard a voice from out in ...,1,8,1,13,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
4628,@savage_esquire That's unfuckingbelievable.\nI...,1,109,51,73,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
4629,@Jomboy_ Doesn’t want to pull anymore Hammys\n...,1,2,23,4,71.75678,149.802163,89.247963,73.500359,63.934939,102.471


In [268]:
genuine_train

,tweet_mod,label,no_of_hashtags,no_of_emojis,no_of_urls,no_of_hashtags_avg,no_of_hashtags_std,no_of_urls_avg,no_of_urls_std,no_of_emojis_avg,no_of_emojis_std
0,RT @CarnivalCruise: 🎉 Are you ready to see wha...,0,152,73,207,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
1,RT @realDonaldTrump: THANK YOU #RNC2020! https...,0,36,69,93,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
2,A family fears they may have been cheated out ...,0,60,28,173,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
3,"RT @elonmusk: Aloha, welcome back from space 💫...",0,9,8,178,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
4,So amazing to see what @yael and @CitizenApp a...,0,84,11,112,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
...,...,...,...,...,...,...,...,...,...,...,...
3587,@JOHNSHIP99 @gunsnroses @Aerosmith It was.\nRT...,0,74,8,75,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
3588,I agree. Go home at sundown. No more rioting a...,0,20,21,45,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
3589,@tomilahren imagine if Humayun Khan died in Be...,0,34,7,63,71.75678,149.802163,89.247963,73.500359,63.934939,102.471
3590,"This #LaborDayWeekend, binge-watch the East Si...",0,129,4,236,71.75678,149.802163,89.247963,73.500359,63.934939,102.471


In [269]:
bots_train['no_of_hashtags_avg'] = bots_train['no_of_hashtags'].mean()
genuine_train['no_of_hashtags_avg'] = genuine_train['no_of_hashtags'].mean()

bots_train['no_of_hashtags_std'] = bots_train['no_of_hashtags'].std()
genuine_train['no_of_hashtags_std'] = genuine_train['no_of_hashtags'].std()

In [270]:
bots_test['no_of_hashtags_avg'] = bots_test['no_of_hashtags'].mean()
genuine_test['no_of_hashtags_avg'] = genuine_test['no_of_hashtags'].mean()

bots_test['no_of_hashtags_std'] = bots_test['no_of_hashtags'].std()
genuine_test['no_of_hashtags_std'] = genuine_test['no_of_hashtags'].std()

In [271]:
bots_train['no_of_emojis_avg'] = bots_train['no_of_emojis'].mean()
genuine_train['no_of_emojis_avg'] = genuine_train['no_of_emojis'].mean()

bots_train['no_of_emojis_std'] = bots_train['no_of_emojis'].std()
genuine_train['no_of_emojis_std'] = genuine_train['no_of_emojis'].std()

In [272]:
bots_test['no_of_emojis_avg'] = bots_test['no_of_emojis'].mean()
genuine_test['no_of_emojis_avg'] = genuine_test['no_of_emojis'].mean()

bots_test['no_of_emojis_std'] = bots_test['no_of_emojis'].std()
genuine_test['no_of_emojis_std'] = genuine_test['no_of_emojis'].std()

In [273]:

bots_train['no_of_urls_avg'] = bots_train['no_of_urls'].mean()
genuine_train['no_of_urls_avg'] = genuine_train['no_of_urls'].mean()

bots_train['no_of_urls_std'] = bots_train['no_of_urls'].std()
genuine_train['no_of_urls_std'] = genuine_train['no_of_urls'].std()

In [274]:

bots_test['no_of_urls_avg'] = bots_test['no_of_urls'].mean()
genuine_test['no_of_urls_avg'] = genuine_test['no_of_urls'].mean()

bots_test['no_of_urls_std'] = bots_test['no_of_urls'].std()
genuine_test['no_of_urls_std'] = genuine_test['no_of_urls'].std()

In [275]:
bots_train

,tweet_mod,label,no_of_hashtags,no_of_emojis,no_of_urls,no_of_hashtags_avg,no_of_hashtags_std,no_of_urls_avg,no_of_urls_std,no_of_emojis_avg,no_of_emojis_std
0,RT @VonteThePlug: Yeah but he ain’t got one ha...,1,6,108,57,78.993306,173.684091,83.088318,72.950892,54.775858,92.500138
1,https://t.co/KWnKV9F0fw\n,1,0,0,1,78.993306,173.684091,83.088318,72.950892,54.775858,92.500138
2,RT @toadmeister: If the BBC is investing £100 ...,1,33,21,92,78.993306,173.684091,83.088318,72.950892,54.775858,92.500138
3,@BrightskinMacky 😂😂😂\nBone what??? 😂😂😂 RT @sun...,1,16,93,14,78.993306,173.684091,83.088318,72.950892,54.775858,92.500138
4,"Milano pigliatutto. Dopo l’ex Banco Popolare, ...",1,124,16,141,78.993306,173.684091,83.088318,72.950892,54.775858,92.500138
...,...,...,...,...,...,...,...,...,...,...,...
4626,Love &amp; Entrepreneurship: Relationship Ther...,1,30,22,161,78.993306,173.684091,83.088318,72.950892,54.775858,92.500138
4627,@C130Matt I think I heard a voice from out in ...,1,8,1,13,78.993306,173.684091,83.088318,72.950892,54.775858,92.500138
4628,@savage_esquire That's unfuckingbelievable.\nI...,1,109,51,73,78.993306,173.684091,83.088318,72.950892,54.775858,92.500138
4629,@Jomboy_ Doesn’t want to pull anymore Hammys\n...,1,2,23,4,78.993306,173.684091,83.088318,72.950892,54.775858,92.500138


In [276]:
genuine_train

,tweet_mod,label,no_of_hashtags,no_of_emojis,no_of_urls,no_of_hashtags_avg,no_of_hashtags_std,no_of_urls_avg,no_of_urls_std,no_of_emojis_avg,no_of_emojis_std
0,RT @CarnivalCruise: 🎉 Are you ready to see wha...,0,152,73,207,62.42706,111.047802,97.18931,73.454395,75.743318,112.969352
1,RT @realDonaldTrump: THANK YOU #RNC2020! https...,0,36,69,93,62.42706,111.047802,97.18931,73.454395,75.743318,112.969352
2,A family fears they may have been cheated out ...,0,60,28,173,62.42706,111.047802,97.18931,73.454395,75.743318,112.969352
3,"RT @elonmusk: Aloha, welcome back from space 💫...",0,9,8,178,62.42706,111.047802,97.18931,73.454395,75.743318,112.969352
4,So amazing to see what @yael and @CitizenApp a...,0,84,11,112,62.42706,111.047802,97.18931,73.454395,75.743318,112.969352
...,...,...,...,...,...,...,...,...,...,...,...
3587,@JOHNSHIP99 @gunsnroses @Aerosmith It was.\nRT...,0,74,8,75,62.42706,111.047802,97.18931,73.454395,75.743318,112.969352
3588,I agree. Go home at sundown. No more rioting a...,0,20,21,45,62.42706,111.047802,97.18931,73.454395,75.743318,112.969352
3589,@tomilahren imagine if Humayun Khan died in Be...,0,34,7,63,62.42706,111.047802,97.18931,73.454395,75.743318,112.969352
3590,"This #LaborDayWeekend, binge-watch the East Si...",0,129,4,236,62.42706,111.047802,97.18931,73.454395,75.743318,112.969352


In [277]:
bots_train_new = bots_train[['no_of_hashtags_avg', 'no_of_hashtags_std', 'no_of_emojis_avg', 'no_of_emojis_std', 'no_of_urls_avg', 'no_of_urls_std']]

In [278]:
bots_test_new = bots_test[['no_of_hashtags_avg', 'no_of_hashtags_std', 'no_of_emojis_avg', 'no_of_emojis_std', 'no_of_urls_avg', 'no_of_urls_std']]

In [279]:
bots_train_new

,no_of_hashtags_avg,no_of_hashtags_std,no_of_emojis_avg,no_of_emojis_std,no_of_urls_avg,no_of_urls_std
0,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
1,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
2,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
3,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
4,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
...,...,...,...,...,...,...
4626,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
4627,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
4628,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
4629,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892


In [280]:
genuine_train_new = genuine_train[['no_of_hashtags_avg', 'no_of_hashtags_std', 'no_of_emojis_avg', 'no_of_emojis_std', 'no_of_urls_avg', 'no_of_urls_std']]

In [281]:
genuine_test_new = genuine_test[['no_of_hashtags_avg', 'no_of_hashtags_std', 'no_of_emojis_avg', 'no_of_emojis_std', 'no_of_urls_avg', 'no_of_urls_std']]

In [282]:
hue_df = pd.concat([bots_train_new, genuine_train_new], axis = 0)

In [283]:
hue_df_te = pd.concat([bots_test_new, genuine_test_new], axis = 0)

In [284]:
hue_df

,no_of_hashtags_avg,no_of_hashtags_std,no_of_emojis_avg,no_of_emojis_std,no_of_urls_avg,no_of_urls_std
0,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
1,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
2,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
3,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
4,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
...,...,...,...,...,...,...
3587,62.427060,111.047802,75.743318,112.969352,97.189310,73.454395
3588,62.427060,111.047802,75.743318,112.969352,97.189310,73.454395
3589,62.427060,111.047802,75.743318,112.969352,97.189310,73.454395
3590,62.427060,111.047802,75.743318,112.969352,97.189310,73.454395


In [285]:
final_train = pd.concat([tr_prof, hue_df], axis = 1)
final_test = pd.concat([te_prof, hue_df_te],axis = 1)

In [286]:
features = ['default_profile', 'favourites_count', 'followers_count', 'friends_count', 'geo_enabled', 'listed_count', 'profile_use_background_image', 'statuses_count', 'verified', 'tweet_freq', 'followers_growth_rate', 'friends_growth_rate', 'favourites_growth_rate', 'listed_growth_rate', 'followers_friends_ratio', 'screen_name_length', 'num_digits_in_screen_name', 'name_length', 'num_digits_in_name', 'description_length']

In [287]:
final_train

,default_profile,favourites_count,followers_count,friends_count,geo_enabled,listed_count,profile_use_background_image,statuses_count,verified,tweet_freq,...,name_length,num_digits_in_name,description_length,label,no_of_hashtags_avg,no_of_hashtags_std,no_of_emojis_avg,no_of_emojis_std,no_of_urls_avg,no_of_urls_std
0,1,729,13324,647,0,44,1,103,0,0.001341,...,18,0,104,1,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
1,1,0,4,66,0,0,1,0,0,0.000000,...,16,0,58,1,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
2,1,2224,13120,1406,0,235,1,44434,0,0.526569,...,14,0,58,1,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
3,0,382,2829,789,0,31,1,211695,0,1.856941,...,6,0,50,1,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
4,1,1419,648,513,0,7,1,542,0,0.006014,...,15,0,43,1,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3587,0,2431,14141,769,0,211,1,15400,1,0.143579,...,16,0,85,0,62.427060,111.047802,75.743318,112.969352,97.189310,73.454395
3588,1,5774,127,540,1,0,1,4053,0,0.245116,...,5,0,128,0,62.427060,111.047802,75.743318,112.969352,97.189310,73.454395
3589,0,88,8676,131,0,266,1,5721,1,0.055864,...,15,0,13,0,62.427060,111.047802,75.743318,112.969352,97.189310,73.454395
3590,0,25867,34008,15566,1,719,1,11177,1,0.096061,...,15,0,158,0,62.427060,111.047802,75.743318,112.969352,97.189310,73.454395


In [288]:
final_test

,default_profile,favourites_count,followers_count,friends_count,geo_enabled,listed_count,profile_use_background_image,statuses_count,verified,tweet_freq,...,name_length,num_digits_in_name,description_length,label,no_of_hashtags_avg,no_of_hashtags_std,no_of_emojis_avg,no_of_emojis_std,no_of_urls_avg,no_of_urls_std
0,1,42684,16596,16944,0,1,1,49757,0,2.231056,...,21,0,158,1,75.924290,171.958785,55.820189,99.133939,85.290221,79.954871
1,1,0,9,543,0,0,1,2,0,0.000132,...,12,0,146,1,75.924290,171.958785,55.820189,99.133939,85.290221,79.954871
2,1,25891,688,748,0,0,1,5232,0,0.057686,...,15,0,56,1,75.924290,171.958785,55.820189,99.133939,85.290221,79.954871
3,0,0,1334,1812,0,4,1,670,0,0.006921,...,12,0,73,1,75.924290,171.958785,55.820189,99.133939,85.290221,79.954871
4,1,14,2,48,0,0,1,16,0,0.000914,...,9,0,1,1,75.924290,171.958785,55.820189,99.133939,85.290221,79.954871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,0,2211,239402,143,1,2929,1,19881,1,0.172938,...,44,0,121,0,60.037106,100.325708,83.174397,109.144592,100.677180,76.072576
535,0,41333,19700,1298,0,328,1,87946,1,0.936722,...,10,0,95,0,60.037106,100.325708,83.174397,109.144592,100.677180,76.072576
536,0,11853,19933,5349,1,523,1,34705,1,0.301431,...,18,0,116,0,60.037106,100.325708,83.174397,109.144592,100.677180,76.072576
537,1,33227,13002,3243,1,254,1,19252,1,0.166423,...,11,0,121,0,60.037106,100.325708,83.174397,109.144592,100.677180,76.072576


In [289]:
all = pd.concat([final_train, final_test], axis = 0)

In [290]:
all

,default_profile,favourites_count,followers_count,friends_count,geo_enabled,listed_count,profile_use_background_image,statuses_count,verified,tweet_freq,...,name_length,num_digits_in_name,description_length,label,no_of_hashtags_avg,no_of_hashtags_std,no_of_emojis_avg,no_of_emojis_std,no_of_urls_avg,no_of_urls_std
0,1,729,13324,647,0,44,1,103,0,0.001341,...,18,0,104,1,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
1,1,0,4,66,0,0,1,0,0,0.000000,...,16,0,58,1,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
2,1,2224,13120,1406,0,235,1,44434,0,0.526569,...,14,0,58,1,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
3,0,382,2829,789,0,31,1,211695,0,1.856941,...,6,0,50,1,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
4,1,1419,648,513,0,7,1,542,0,0.006014,...,15,0,43,1,78.993306,173.684091,54.775858,92.500138,83.088318,72.950892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,0,2211,239402,143,1,2929,1,19881,1,0.172938,...,44,0,121,0,60.037106,100.325708,83.174397,109.144592,100.677180,76.072576
535,0,41333,19700,1298,0,328,1,87946,1,0.936722,...,10,0,95,0,60.037106,100.325708,83.174397,109.144592,100.677180,76.072576
536,0,11853,19933,5349,1,523,1,34705,1,0.301431,...,18,0,116,0,60.037106,100.325708,83.174397,109.144592,100.677180,76.072576
537,1,33227,13002,3243,1,254,1,19252,1,0.166423,...,11,0,121,0,60.037106,100.325708,83.174397,109.144592,100.677180,76.072576


In [291]:
from sklearn import svm, tree
import xgboost
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

In [292]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix
#all[features] = scaler.fit_transform(all[features])
X_train, X_test, y_train, y_test = train_test_split(all[features], all['label'], test_size=0.5, random_state=1, stratify=all['label'])

In [293]:
xgb = xgboost.XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print('Accuracy: %.4f' % accuracy_score(y_test, y_pred))

Accuracy: 0.8203
